In [1]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Convolution2D,Dense,MaxPool2D,Activation,Dropout,Flatten
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.layers import BatchNormalization
import os 
import pandas as pd
import plotly.graph_objs as go
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

In [2]:
!pip install opendatasets
import opendatasets as od
od.download('https://www.kaggle.com/datasets/amrragababdelaziz/dermnet-and-skin-disease-9-classes')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: gaurav065
Your Kaggle Key: ··········


100%|██████████| 3.51G/3.51G [00:33<00:00, 113MB/s]


In [3]:
train ='/content/dermnet-and-skin-disease-9-classes/Merged Dataset/Train'
validation_dir = '/content/dermnet-and-skin-disease-9-classes/Merged Dataset/test'

In [4]:
import glob
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count

In [5]:
train_samples =get_files(train)
num_classes=len(glob.glob(train+"/*"))
test_samples=get_files(validation_dir)
print(num_classes,"Classes")
print(train_samples,"Train images")
print(test_samples,"Test images")

9 Classes
26642 Train images
2389 Test images


In [6]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)
img_width,img_height =224,224
input_shape=(img_width,img_height,3)
batch_size =64
train_generator =train_datagen.flow_from_directory(train,
                                target_size=(img_width,img_height),batch_size=batch_size)
test_generator=test_datagen.flow_from_directory(validation_dir,shuffle=True,target_size=(img_width,img_height),batch_size=batch_size)

Found 26642 images belonging to 9 classes.
Found 2389 images belonging to 9 classes.


In [7]:
from tensorflow import keras
from keras import layers
model = keras.Sequential([
    layers.Input((224, 224, 3)),
    
    layers.Conv2D(16, 3, activation="relu"),
    layers.BatchNormalization(), # Regularization
    layers.MaxPooling2D(), 
    
    layers.Conv2D(32, 3, activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, 3, activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    
    layers.Dense(64, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.2), # Regularization
    layers.Dense(9, activation="softmax"),
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)
history = model.fit(train_generator,validation_data=test_generator,epochs=70,shuffle=True)

Epoch 1/70
833/833 [==============================] - 461s 553ms/step - loss: 1.6144 - accuracy: 0.4264 - val_loss: 2.0515 - val_accuracy: 0.2993
Epoch 2/70
833/833 [==============================] - 463s 556ms/step - loss: 1.4260 - accuracy: 0.4877 - val_loss: 1.8526 - val_accuracy: 0.3495
Epoch 3/70
833/833 [==============================] - 454s 545ms/step - loss: 1.3364 - accuracy: 0.5197 - val_loss: 1.8234 - val_accuracy: 0.3550
Epoch 4/70
833/833 [==============================] - 446s 535ms/step - loss: 1.2767 - accuracy: 0.5444 - val_loss: 1.6143 - val_accuracy: 0.4161
Epoch 5/70
833/833 [==============================] - 449s 539ms/step - loss: 1.2239 - accuracy: 0.5682 - val_loss: 1.8875 - val_accuracy: 0.3524
Epoch 6/70
833/833 [==============================] - 438s 525ms/step - loss: 1.1682 - accuracy: 0.5878 - val_loss: 1.6825 - val_accuracy: 0.4035
Epoch 7/70
833/833 [==============================] - 432s 518ms/step - loss: 1.1182 - accuracy: 0.6072 - val_loss: 1.5983 -

In [ ]:
model.save('CNN_model.h5')